---
# **機械学習特論　第5回課題**
## **【k近傍法・サポートベクターマシン】アヤメ・手書き数字・衣類に関するデータセットの分類**
---

In [1]:
# ライブラリの読み込み
# %pip install --upgrade pip
# %pip install numpy
# %pip install matplotlib
# %pip install pandas
# %pip install openpyxl
# %pip install sympy
# %pip install scipy
# %pip install re
# %pip install jaconv
# %pip install scikit-learn
# %pip install statsmodels
# %pip install seaborn
# %pip install pmdarima
# %pip install kneed
# %pip install scikit-learn-intelex
from matplotlib.font_manager import FontProperties
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import sympy as sp
import scipy.stats as stats
import re
import jaconv 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import itertools
import matplotlib.dates as mdates
import pmdarima as pm
from IPython.display import clear_output
from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from kneed import KneeLocator
from sklearn.decomposition import PCA
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import time
from matplotlib.ticker import LogLocator, LogFormatter
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
from sklearnex import patch_sklearn

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='seaborn')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")


### 1. 課題
#### (1) データセット
##### ・Irisのデータセット  
　簡単のため，今回は2種類の分類のみを扱う．
##### ・MNIST（手書き数字）のデータセット
　0～9の数字が用意されており，これらを10個に分類する．データ数が多く，学習モデルのテストに用いられることも多い．
##### ・Fashion-MNIST（衣類）のデータセット
　衣類の写真が用意されたデータセットである．こちらも画像系の学習モデルのテストとして用いられることが多い．

In [2]:
# sklearnデータセットに収録されたiris(アヤメ)のデータセットをロードしてデータフレームを作成
def load_iris_data():
    data = load_iris()
    x = pd.DataFrame(data["data"],columns=data["feature_names"])[50:150]
    y = pd.DataFrame(data["target"],columns=["target"])[50:150] # species 1 2 を抽出
    x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return x_train, x_test, y_train, y_test 

# 手書き文字のデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_mnist_data():
    data = fetch_openml('mnist_784', version=1)
    x = np.array(data['data'].astype(np.float32))
    y = np.array(data['target'].astype(np.int32))
    x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return x_train, x_test, y_train, y_test 

# Fashion-MNISTデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_fashion_mnist_data():
    data = fetch_openml('Fashion-MNIST')
    x = np.array(data['data'].astype(np.float32))
    y = np.array(data['target'].astype(np.int32))
    x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return x_train, x_test, y_train, y_test 

# 一括処理のためにデータセットの辞書を作成
dataset = {'iris': load_iris_data(), 'mnist': load_mnist_data(), 'fashon-mnist': load_fashion_mnist_data()}

#### (2) 分類モデル
##### ・k近傍法
　説明変数に関する空間上の距離に従って，分類する手法である．ここで，$ k $は近くにある$ k $個のデータに従いクラスを決定することを意味する．$ k $が小さいとノイズに弱くなるが，大きすぎると精度が悪くなることが知られている．
##### ・サポートベクターマシン（SVM）
　対象データを2クラスに識別する手法である．一方のクラスのサポートベクターから境界への距離(マージン)と，もう一方のクラスのサポートベクターから境界への距離(マージン)が最大になる境界を採用してクラスを識別する．誤りなく識別できる場合はハードマージン法を，できない場合はソフトマージン法を利用する．

#### (3) 条件
##### 1) k近傍法：$ k=3$
##### 2) SVM：kernel="linear", c=1, 標準化
##### 3) SVM：kernel="rbf", c=1, 標準化

In [4]:
# 一括処理のためにモデルの辞書を作成
model = {
    # k近傍法のモデル
    'kNN(k=3)':
    KNeighborsClassifier(n_neighbors=3, # k を指定 (デフォルトは 5)
                        weights='uniform',  # 距離を考慮しない(uniform:デフォルト)、する(distance)
                        algorithm='auto', # 近傍点計算アルゴリズム (auto:デフォルト,ball_tree,kd_tree,brute)
                        leaf_size=30,  # ball_tree,kd_tree指定時のリーフサイズの設定 (デフォルトは 30)
                        p=2),  # 距離計算の次元 (2:デフォルト、1)
    # svm (kernel="linear", C=1) のモデル
    'SVC(kernel="linear", C=1)':
    svm.SVC(kernel="linear", C=1, max_iter=100000, verbose=True, random_state=1),
    # svm (kernel="rbf", C=1) のモデル
    'SVC(kernel="rbf", C=1)':
    svm.SVC(kernel="rbf", C=1, max_iter=100000, verbose=True, random_state=1),
}

In [5]:
# 辞書に格納したデータセットそれぞれについて性能を確認
for dataset_key in dataset.keys():
    x_train, x_test, y_train, y_test = dataset[dataset_key]
    print(f'## dataset:{dataset_key} ',
        f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # データ標準化なしで性能を測定
    print('# no scaling')
    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

    # データを標準化
    print('# with scaling')
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

## dataset:iris  x_train:80 x_test:20 y_train:80 y_test:20
# no scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.9375 test_data: 0.95
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.9625 test_data: 0.95
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.975 test_data: 0.95
# with scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.95 test_data: 0.95
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.9625 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.975 test_data: 0.95
## dataset:mnist  x_train:56000 x_test:14000 y_train:56000 y_test:14000
# no scaling
dataset:mnist model:kNN(k=3) accuracy_score: train_data: 0.98586 test_data: 0.97229
[LibSVM]dataset:mnist model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.88802 test_data: 0.84421
[LibSVM]dataset:mnist model:SVC(kernel="rbf", C=1) accuracy_score: train

| モデル       | パラメータ | Iris | MNIST | Fashion-MNIST |
|--------------|------------|------|-------|---------------|
| kNN      | $k=3$ | 0.95 | 0.94629 | 0.85586 |
| SVM      | kernel="linear", c=1, 標準化 | 1.00 | 0.92214 | 0.81029 |
| SVM      | kernel="rbf", c=1, 標準化 | 0.95 | 0.96607  | 0.89607 |


標準化をしないと警告文が出力されることがあった．また，データセットとモデルによって標準化を行った方が精度が良くなるものと，行わない方が精度が良くなるものの両方が確認できた．

### 2. 最適パラメータの探索

In [ ]:
# 警告を無視
warnings.filterwarnings("ignore")

from sklearnex import patch_sklearn
patch_sklearn()

# グリッドサーチを用いた最適パラメータの探索
for dataset_key in dataset.keys():
    x_train, x_test, y_train, y_test = dataset[dataset_key]
    print(f'## dataset:{dataset_key} ')

    # データを標準化
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # k近傍法のグリッドサーチ
    if dataset_key == 'iris':
        param_grid_knn = {
            'n_neighbors': range(1, 81),
            'p': range(1, 11)
        }
        knn = KNeighborsClassifier(weights='uniform', algorithm='auto', leaf_size=30)
        grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
        grid_search_knn.fit(x_train, np.array(y_train).ravel())

        best_knn_params = grid_search_knn.best_params_
        best_knn_score = grid_search_knn.best_score_
        print(f"kNN: Best score: {best_knn_score: 0.5}, Best k: {best_knn_params['n_neighbors']}, Best p: {best_knn_params['p']}")

    # SVMのグリッドサーチ
    param_grid_svm = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }
    svm_model = svm.SVC(max_iter=100000, random_state=1)
    grid_search_svm = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
    grid_search_svm.fit(x_train, np.array(y_train).ravel())

    best_svm_params = grid_search_svm.best_params_
    best_svm_score = grid_search_svm.best_score_
    print(f"SVM: Best score: {best_svm_score: 0.5}, Best kernel: {best_svm_params['kernel']}, Best C: {best_svm_params['C']}")


k近傍法のパラメータは，kの値とp（ミンコフスキー距離の指数部分：距離の種類）を変更し，SVMはカーネルの種類（線形，多項式，ガウシアン，シグモイド）とCの値（誤分類に対するペナルティ：ハード／ソフトマージン）の組み合わせをグリッドサーチを用いて探索し，最適なパラメータを探索することを試みた．しかし，探索時間がかかりすぎて終わらなかったため，少し妥協した探索プログラムに改良した．

In [3]:
# 警告を無視
warnings.filterwarnings("ignore")

from sklearnex import patch_sklearn
patch_sklearn()

# グリッドサーチをfor文に変換した最適パラメータの探索
for dataset_key in dataset.keys():
    x_train, x_test, y_train, y_test = dataset[dataset_key]
    print(f'## dataset: {dataset_key}')

    # データを標準化
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # k近傍法の探索
    if dataset_key == 'iris':
        best_knn_score = 0
        best_k, best_p = None, None
        for k in range(1, 81):
            for p in range(1, 11):
                knn = KNeighborsClassifier(n_neighbors=k, p=p, weights='uniform', algorithm='auto', leaf_size=30)
                knn.fit(x_train, np.array(y_train).ravel())
                score = knn.score(x_test, np.array(y_test).ravel())
                if score > best_knn_score:
                    best_knn_score = score
                    best_k, best_p = k, p
                if best_knn_score == 1.0:
                    break
            if best_knn_score == 1.0:
                break
        print(f"kNN: Best score: {best_knn_score:0.5}, Best k: {best_k}, Best p: {best_p}")

    # SVMの探索
    best_svm_score = 0
    best_kernel, best_C = None, None
    for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
        svm_model = svm.SVC(kernel=kernel, C=1, max_iter=100000, random_state=1)
        svm_model.fit(x_train, np.array(y_train).ravel())
        score = svm_model.score(x_test, np.array(y_test).ravel())
        if score > best_svm_score:
            best_svm_score = score
            best_kernel = kernel
        if best_svm_score == 1.0:
            break
    best_svm_score = 0
    for c in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
        svm_model = svm.SVC(kernel=best_kernel, C=c, max_iter=100000, random_state=1)
        svm_model.fit(x_train, np.array(y_train).ravel())
        score = svm_model.score(x_test, np.array(y_test).ravel())
        if score > best_svm_score:
            best_svm_score = score
            best_C = c
        if best_svm_score == 1.0:
            break
    print(f"SVM: Best score: {best_svm_score:0.5}, Best kernel: {best_kernel}, Best C: {best_C}")


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## dataset: iris
kNN: Best score: 0.95, Best k: 1, Best p: 1
SVM: Best score: 1.0, Best kernel: linear, Best C: 0.1
## dataset: mnist
SVM: Best score: 0.97207, Best kernel: rbf, Best C: 100
## dataset: fashon-mnist
SVM: Best score: 0.90736, Best kernel: rbf, Best C: 10


　単純なデータセットに対しては，SVMの線形カーネルのソフトマージンが有効であることが分かった．逆にソフトマージンでも正しく分類できるほどデータが単調であった（分類しやすかった）と考えられる．
　MNISTのような複雑なデータセットに対しては，ガウシアンカーネルとハードマージンの組み合わせが有効であることが分かった．これは非線形なデータが多く含まれていることに起因すると考えられる．特にMNISTの方はかなり精度の良いモデルが構築できた．

### 感想  
　カーネルやパラメータの違いで精度に大きく影響することが確認できた．今後，深層学習を行ったり，目的を達成するための機械学習モデルを構成する際も，意思を持ってパラメータを選択できるようになるためにも，この辺りのパラメータの意味には注意したいと感じた．